# Train a first Transformer Model

Use the pretrained bert_base_cased model

Tutorial:
https://huggingface.co/docs/transformers/training

In [1]:
MODEL = 'bert-base-cased'

In [2]:
import pandas as pd

train_data = pd.read_pickle('../data/05a_Authors_Train_Set.pkl')
validate_data = pd.read_pickle('../data/05b_Authors_Validate_Set.pkl')
test_data = pd.read_pickle('../data/05c_Authors_Test_Set.pkl')

In [3]:
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

d = {
    'train': Dataset.from_dict({'label': train_data['label'].astype('int32'), 'text': train_data['message']}),
    'val': Dataset.from_dict({'label': validate_data['label'].astype('int32'), 'text': validate_data['message']}),
    'test': Dataset.from_dict({'label': test_data['label'].astype('int32'), 'text': test_data['message']})
}

dataset_dict = DatasetDict(d)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

  0%|          | 0/48 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["val"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=42)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Evaluate

In [7]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

FileNotFoundError: Couldn't find a module script at /Users/fabian/Developer/HPI/02 Trends AI:DL/03 Data Clustering/accuracy/accuracy.py. Module 'accuracy' doesn't exist on the Hugging Face Hub either.

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir='test_trainer_small', evaluation_strategy='epoch')

### Training

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/fabian/Library/Python/3.8/lib/python/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 108342570


  0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: 